<a href="https://colab.research.google.com/github/Michael-Jimenez-C/CieCom2/blob/main/proyecto_CII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import sympy as sp
import numpy as np
sp.init_session()

IPython console for SymPy 1.11.1 (Python 3.9.16-64-bit) (ground types: python)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.11.1/



In [2]:
class Cuarto:
  def __init__(self,capacitancia,temperatura,fuente=0,Rfuente=1):
    t=sp.symbols("t")
    self.temperatura=temperatura
    self.adyacentes=[]#[(nodo,resistencia)]
    self.capacitancia=capacitancia
    self.fuente=fuente
    self.Rfuente=Rfuente
  def eq(self):
    equ=0
    if type(self.fuente)!=list:
      equ+=(self.fuente-self.temperatura if self.fuente!=0 else 0)/self.Rfuente
    else:
      for i,j in zip(self.fuente,self.Rfuente):
        equ+=(i-self.temperatura if i!=0 else 0)/j
    for i in self.adyacentes:
      equ+=(i[0].temperatura-self.temperatura)*sp.Rational(1,i[1])
    return sp.simplify(equ*sp.Rational(1,self.capacitancia))

In [3]:
t,s=sp.symbols("t s")
T1,T2,T3=sp.symbols("T_{1} T_{2} T_{3}")
A1=Cuarto(10,T1,sp.symbols("T_{i1}"))
B1=Cuarto(5,T2,0)
C1=Cuarto(10,T3,0)

A1.adyacentes=[(B1,15)]
B1.adyacentes=[(A1,11),(C1,20)]
C1.adyacentes=[(A1,20)]

nodos=[A1,B1,C1]

In [4]:
nodos[1].eq()

T_{1}/55 - 31*T_{2}/1100 + T_{3}/100

In [165]:

variables=sp.symbols("T_{1} T_{2} T_{3}")
entradas=[sp.symbols("T_{i1}")]
A=[]
B=[]
for i in range(len(nodos)):
  A.append([])
  B.append([])
  eq=nodos[i].eq()
  for j in range(len(variables)):
    A[-1].append(eq.coeff(variables[j]))
  for j in range(len(entradas)):
    B[-1].append(eq.coeff(entradas[j]))

A=sp.Matrix(A)
B=sp.Matrix(B)
C=sp.Matrix([[1 for i in range(A.shape[0])]])
A,B

⎛⎡-8/75  1/150    0   ⎤        ⎞
⎜⎢                    ⎥  ⎡1/10⎤⎟
⎜⎢       -31          ⎥  ⎢    ⎥⎟
⎜⎢1/55   ────   1/100 ⎥, ⎢ 0  ⎥⎟
⎜⎢       1100         ⎥  ⎢    ⎥⎟
⎜⎢                    ⎥  ⎣ 0  ⎦⎟
⎝⎣1/200    0    -1/200⎦        ⎠

In [ ]:
class Edificio:
  pass

r

In [171]:
def eAt(A,t):
  T,D=sp.N(A,2).diagonalize()
  return T*sp.exp(D*t)*T.inv()

In [173]:
eAt(A,t)

⎡                    -0.108165144920349⋅t                       -0.02683982253
⎢ 0.982264496412163⋅ℯ                     + 0.0162186547024561⋅ℯ              
⎢                                                                             
⎢                      -0.108165144920349⋅t                      -0.0268398225
⎢ - 0.217384798860847⋅ℯ                     + 0.194218234026722⋅ℯ             
⎢                                                                             
⎢                     -0.108165144920349⋅t                        -0.026839822
⎣- 0.047579113397671⋅ℯ                     - 0.00371062623226229⋅ℯ            

07465⋅t                        -0.00484947115182877⋅t                         
        + 0.00151684888538071⋅ℯ                         - 0.0818747914509728⋅ℯ
                                                                              
307465⋅t                       -0.00484947115182877⋅t                         
         + 0.0231665648341251⋅ℯ                    

In [180]:
x0=sp.Matrix([[0],[0],[0]])
u=sp.Matrix([[5]])

⎡  ⎛             t        -t⎞             t        -t⎤
⎢- ⎝1.0⋅t - 0.5⋅ℯ  + 0.5⋅ℯ  ⎠⋅θ(t) + 0.5⋅ℯ  - 0.5⋅ℯ  ⎥
⎢                                                    ⎥
⎢            ⎛          -t⎞             -t           ⎥
⎣          - ⎝-t + 1 - ℯ  ⎠⋅θ(t) + 1.0⋅ℯ             ⎦

In [197]:
t,s=sp.symbols("t s")

A=sp.Matrix([[1,1],[0,-1]])
B=sp.Matrix([[0],[1]])
x0=sp.Matrix([[0],[1]])
u=sp.Matrix([[sp.Heaviside(t)*t]])

In [198]:
def solucionTiempo(A,B,x0,u):
  t=sp.symbols("t")
  EAT=eAt(A,t)
  tau=sp.symbols("\tau") 
  return EAT*x0+sp.integrate(EAT.subs(t,tau)*B*u.subs(t,t-tau),(tau,0,t))

In [194]:
def solucionFrecuencia(A,B,x0,u):
  t,s=sp.symbols("t s")
  U=sp.laplace_transform(u,t,s,noconds=True)
  Y=(s*sp.eye(A.shape[0])-A).inv()@(x0+B*U)
  sol=sp.inverse_laplace_transform(sp.expand(Y),s,t)
  return sol


In [212]:
sol=solucionFrecuencia(A,B,x0,u)
sol
sp.N(sol[0].subs(t,10))

u Matrix([[t*Heaviside(t)]])
y Matrix([[(1 + s**(-2))/(s**2 - 1)], [(1 + s**(-2))/(s + 1)]])


22016.4657494068

In [213]:
sol=solucionTiempo(A,B,x0,u)
sol
sp.N(sol[0].subs(t,10))

22016.4657494068

In [209]:
sp.plot(sol[0],0,100)